<a href="https://colab.research.google.com/github/jordantangy/Final-Project/blob/main/plot_embedder_last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# uploaded

import pandas as pd
import numpy as np
import csv

data = pd.read_csv('/content/movie_plots_revisited.csv')
truncat = data[:34000]
truncat.values[0][4]

7.4

In [91]:
import tensorflow_hub as hub
import tensorflow as tf
import torch

model = "https://tfhub.dev/google/universal-sentence-encoder/4"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [92]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [93]:
x = truncat.drop(['id','poster','title','year','rating','genre','action','adventure','animation','comedy','crime','drama','fantasy','horror','mystery','romance','sci-fi','short','thriller'], axis = 1)
x = x.to_numpy()

In [94]:
y = truncat.drop(['id','poster','title','year','rating','genre','plot'], axis = 1)
y = y.to_numpy()

In [95]:
x_train, x_test = x[:25000,:], x[25000:,:]
y_train, y_test = y[:25000,:], y[25000:,:]

In [98]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max')
model = tf.keras.Sequential()
model.add(hub_layer)

model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dense(128, activation='relu'))

model.add(tf.keras.layers.Dense(13,activation='sigmoid'))

model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 512)               256797824 
_________________________________________________________________
dense_138 (Dense)            (None, 16)                8208      
_________________________________________________________________
dropout_92 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_139 (Dense)            (None, 32)                544       
_________________________________________________________________
dropout_93 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_140 (Dense)            (None, 64)                2112      
_________________________________________________________________
dropout_94 (Dropout)         (None, 64)              

In [99]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(name='accuracy')])

In [100]:
x_val = x_train[:5000]
partial_x_train = x_train[5000:]

y_val = y_train[:5000]
partial_y_train = y_train[5000:]

In [101]:
history = model.fit(x_train,
                    y_train,
                    epochs=15,
                    batch_size=16,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks = [callback])

Epoch 1/15
   4/1563 [..............................] - ETA: 1:05:39 - loss: 0.6911 - accuracy: 0.5673

KeyboardInterrupt: ignored

In [82]:
results = model.evaluate(x_test, y_test)

print(results)

282/282 [==============================] - 6s 20ms/step - loss: 0.3459 - accuracy: 0.8496
[0.34589120745658875, 0.849581241607666]


In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()